In [1]:
#import library

import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import plot_model
import keras
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import csv
import seaborn as sns
import os
import subprocess
import sys
from sklearn.preprocessing import MinMaxScaler
import random
import pickle

Using TensorFlow backend.


In [2]:
data = pd.read_csv('HEA_DB_Re_Re.csv') 
t_featurename__=list(data.columns)

targets_data = list(data.columns[-1:])
print(targets_data)

for item in targets_data:
    t_featurename__.remove(item)  


['Phase']


In [3]:
X = data.loc[:,:'Phase']
del X['Phase']
X.head()

,VEC,x,Delta,H,S
0,7.00,0.12,6.35,-22.58,16.01
1,4.38,0.20,11.08,-48.64,13.25
2,7.50,0.29,11.25,-23.00,5.76
3,7.50,0.16,6.84,-30.00,5.76
4,9.20,0.16,9.29,-23.68,13.38


In [4]:
y = data.loc[:,['Phase']]
y.head()

,Phase
0,0
1,0
2,0
3,0
4,0


In [5]:
X_train_total, X_test_origin, y_train_total, y_test = train_test_split(X,y, test_size = 0.3)
X_train_origin, X_val_origin, y_train, y_val = train_test_split(X_train_total,y_train_total, test_size = 0.3)

In [6]:
len(X), len(X_train_total), len(X_test_origin)

(118, 82, 36)

In [7]:
len(X_train_total), len(X_train_origin), len(X_val_origin)

(82, 57, 25)

# Scaled

In [8]:
f = open('scaler', 'wb') 
Scaler = RobustScaler()  
Scaler.fit(X_train_total)
pickle.dump(Scaler,f)
f.close()

In [9]:
S_X_train_total = Scaler.transform(X_train_total)
S_X_train_total[:2,:]

array([[-1.91862955,  4.25      ,  1.48217888, -0.05701754, -3.21824104],
       [-0.07708779,  0.5       ,  0.38466711, -0.66776316,  0.99022801]])

In [10]:
S_X_test = Scaler.transform(X_test_origin)
S_X_test[:2,:]

array([[-0.89079229,  0.5       ,  0.53530599, -1.75219298,  0.25407166],
       [ 0.19700214,  0.25      ,  0.20174849, -0.23903509,  1.04885993]])

In [11]:
S_X_train = Scaler.transform(X_train_origin)
S_X_train[:2,:]

array([[ 0.19700214, -0.25      , -0.01613988,  0.37609649,  0.2019544 ],
       [-0.07708779,  0.5       ,  0.38466711, -0.66776316,  0.99022801]])

In [12]:
S_X_val = Scaler.transform(X_val_origin)
S_X_val[:2,:]

array([[-0.82226981,  1.75      ,  3.01546738, -1.96820175, -1.82410423],
       [-0.06852248, -0.25      , -0.04572966,  0.04057018,  0.09120521]])

In [24]:
from keras import models
from keras import layers

def build_model():
    acth = 'selu'
    acto = 'softmax'
    node = 64
    
    model = models.Sequential()
    model.add(layers.Dense(node,activation=acth))
    model.add(layers.Dense(node,activation=acth))
    model.add(layers.Dense(node,activation=acth))
    model.add(layers.Dense(3, activation=acto))
    
    
    model.compile(optimizer = Adam(lr=0.0001),loss = 'binary_crossentropy',metrics=['accuracy'])
    
    return model

In [25]:
from sklearn.model_selection import ShuffleSplit
X = S_X_train_total
y = y_train_total.values
cv = ShuffleSplit(n_splits=3, test_size=.25, random_state=0)

for train_index, test_index in cv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    model = build_model()
    model.fit(X[train_index],y[train_index],epochs=500,batch_size=128)

TRAIN: [ 3 52  7 66 45 54 26 41 81  4 59 17 60 71 57 34 28 11 55 23 10 31 68 73
  1 32 69 14 50 19 29 61 49 24 35 18  0 80 15  5 63 16 51 20 76 25 37 46
 39 65 58 12 70 36 21  9 78 67 64 47 44] TEST: [27  2 62 43  8 42 40 13 79 53 30 77 22 72 33 75 56 74 48 38  6]


ValueError: Error when checking target: expected dense_12 to have shape (3,) but got array with shape (1,)